**Imports and Setup**

In [7]:
pf_id = 2
# n_objectives = 3
n_scenarios = 5

In [8]:
import functools
import pandas as pd
from ema_workbench import (Model, RealParameter, ScalarOutcome,
                           MultiprocessingEvaluator, ema_logging,
                           Constant, SequentialEvaluator)
from ema_workbench import Samplers
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)

import numpy as np

In [9]:
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.util import ema_logging
from ema_workbench import Policy
from dike_model_function import DikeNetwork
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)

ema_logging.log_to_stderr(ema_logging.INFO)

model, planning_steps = get_model_for_problem_formulation(pf_id) #changes for each file

# def get_do_nothing_policies(l_bol = 0, policy_id = 0):
#     return [Policy('policy %s'%policy_id, **{l.name:l_bol for l in model.levers})]

# policies = get_do_nothing_policies()
# policies
uncertainties = []

In [10]:
# # Load the model:
# function = DikeNetwork()
# # workbench model:
# dike_model = Model('dikesnet', function=function)

# # Specify uncertainties range:
# Real_uncert = {'Bmax': [30, 350], 'pfail': [0, 1]}  # m and [.]
# # breach growth rate [m/day]
# cat_uncert_loc = {'Brate': (1., 1.5, 10)}

# cat_uncert = {f'discount rate {n}': (1.5, 2.5, 3.5, 4.5)
#                 for n in function.planning_steps}

# Int_uncert = {'A.0_ID flood wave shape': [0, 132]}
# # Range of dike heightening:
# dike_lev = {'DikeIncrease': [0, 10]}    # dm

# # Series of five Room for the River projects:
# rfr_lev = [f'{project_id}_RfR' for project_id in range(0, 5)]

# EWS_lev = {'EWS_DaysToThreat': [0, 4]}  # days

# uncertainties = []
# levers = []

# for uncert_name in cat_uncert.keys():
#     categories = cat_uncert[uncert_name]
#     uncertainties.append(CategoricalParameter(uncert_name, categories))

# for uncert_name in Int_uncert.keys():
#     uncertainties.append(IntegerParameter(uncert_name, 
#                                           Int_uncert[uncert_name][0],
#                                           Int_uncert[uncert_name][1]))    

# for dike in function.dikelist:
#     # uncertainties in the form: locationName_uncertaintyName
#     for uncert_name in Real_uncert.keys():
#         name = f"{dike}_{uncert_name}"
#         lower, upper = Real_uncert[uncert_name]
#         uncertainties.append(RealParameter(name, lower, upper))

#     for uncert_name in cat_uncert_loc.keys():
#         name = f"{dike}_{uncert_name}"
#         categories = cat_uncert_loc[uncert_name]
#         uncertainties.append(CategoricalParameter(name, categories))

#     # location-related levers in the form: locationName_leversName
#     for lev_name in dike_lev.keys():
#         for n in function.planning_steps:
#             name = f"{dike}_{lev_name} {n}"
#             levers.append(IntegerParameter(name, dike_lev[lev_name][0],
#                                        dike_lev[lev_name][1]))

# # load uncertainties and levers in dike_model:
# model.uncertainties = uncertainties
# model.levers = levers

# # Problem formulations:
# # Outcomes are all costs, thus they have to minimized:
# direction = ScalarOutcome.MINIMIZE


In [11]:
# don't @ me it has to be set up this way to work
from ema_workbench import Policy
policies = [Policy('policy 0', **{'0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.1_DikeIncrease 0':0,
                                  'A.1_DikeIncrease 1':0,
                                  'A.1_DikeIncrease 2':0,
                                  '1_RfR 0':0,
                                  '1_RfR 1':0,
                                  '1_RfR 2':0,
                                  'A.2_DikeIncrease 0':0,
                                  'A.2_DikeIncrease 1':0,
                                  'A.2_DikeIncrease 2':0,
                                  '2_RfR 0':0,
                                  '2_RfR 1':0,
                                  '2_RfR 2':0,
                                  'A.3_DikeIncrease 0':0,
                                  'A.3_DikeIncrease 1':0,
                                  'A.3_DikeIncrease 2':0,
                                  '3_RfR 0':0,
                                  '3_RfR 1':0,
                                  '3_RfR 2':0,
                                  'A.4_DikeIncrease 0':0,
                                  'A.4_DikeIncrease 1':0,
                                  'A.4_DikeIncrease 2':0,
                                  '4_RfR 0':0,
                                  '4_RfR 1':0,
                                  '4_RfR 2':0,
                                  'A.5_DikeIncrease 0':0,
                                  'A.5_DikeIncrease 1':0,
                                  'A.5_DikeIncrease 2':0,
                                  'EWS_DaysToThreat':0
                                  })]

reference; If we are searching over levers, what values should we assume for the uncertainties? Reference allows us to specify this. If searchover is set to levers, reference should be a Scenario or None, while if searchover is uncertainties, reference should be a Policy or None. In case of a None, the default values of the underlying model are unchanged

In [12]:

with SequentialEvaluator(model) as evaluator:
    results, convergence = evaluator.optimize(nfe=100, searchover='uncertainties', reference=policies, epsilons=[0.001, 0.001, 0.001, 0.001, 0.001])


AssertionError: 

***Save Outputs***

**Narrow Down Using epsilon progress and hypervolume**

In [ ]:
model.outcomes = [ScalarOutcome('Expected Annual Damage', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,5)),
                       ScalarOutcome('Dike Investment Costs', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,2)),
                       ScalarOutcome('RfR Investment Costs', kind=ScalarOutcome.MINIMIZE,
                                    expected_range=(0,1)),
                       ScalarOutcome('Evacuation Costs', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,1)),
                    ScalarOutcome('Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,1))]


In [ ]:
convergence = [HyperVolume(minimum=[0,0,0,0,0], maximum=[1, 1, 1, 1, 1]),
               EpsilonProgress()]

**Robustness**

**Robustness Function**

In [ ]:
import functools
import pandas as pd

def robustness(direction, threshold, data):
    if direction == SMALLER:
        return np.sum(data<=threshold)/data.shape[0]
    else:
        return np.sum(data>=threshold)/data.shape[0]

SMALLER = 'SMALLER'

Expected_Annual_Damage = functools.partial(robustness, SMALLER, 1000) #THOSE NUMBERS NEED TO BE SPECIFIED AGAINS
Dike_Investment_Costs = functools.partial(robustness, SMALLER, 1000)
RfR_Investment_Costs = functools.partial(robustness, SMALLER, 1000)
Evacuation_Costs = functools.partial(robustness, SMALLER, 1000)
Expected_Number_of_Deaths = functools.partial(robustness, SMALLER, 0.000001) #not ok

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(10,10),
                        sharex=True)
axes = [axes[0,0],axes[0,1],axes[1,0],axes[1,1]]                             #axes[1,1]

robustness_funcs = {"Expected Annual Damage": Expected_Annual_Damage,
                    "Dike Investment Costs": Dike_Investment_Costs,
                    "RfR Investment Costs": RfR_Investment_Costs,
                    "Evacuation Costs": Evacuation_Costs,
                    "Expected Number of Deaths": Expected_Number_of_Deaths}

experiments, outcomes = results

for ax, (outcome, value) in zip(axes, outcomes.items()):
    for policy in np.unique(experiments['policy']):
        logical = experiments['policy'] == policy
        data = value[logical]
        
        robustness = []
      
        for i in range(1, data.shape[0]):
            robustness.append(robustness_funcs[outcome](data[0:i]))
        ax.plot(robustness, label=policy)

    ax.set_xlabel("# experiments")
    ax.set_ylabel(outcome)